## 天眼查反爬策略分析

> 接口列表参见: [企业信息相关.ipynb](http://192.168.40.190:8888/notebooks/%E4%BC%81%E4%B8%9A%E4%BF%A1%E6%81%AF%E7%9B%B8%E5%85%B3.ipynb)

### 策略

1. 直接访问API接口无法获取数据
2. 使用token去请求数据,会返回"有毒"(假)数据

### Crack流程

**方法**

1. 查看浏览器中的请求
2. 跟踪js代码
3. 浏览器插件拦截请求,修改cookie
4. 注入js代码，查看运行时的变量值

**Part 1**

每个API请求前,访问`http://www.tianyancha.com/tongji/%.json`,获取解密token,请求结果类似下面内容:
```json
{
    "state":"ok",
    "message":"",
    "data":{
       "name":"1698375",
       "uv":616749,
       "pv":159681,
       "v":"33,102,117,110,99,116,105,111,110,40,110,41,123,100,111,99,117,109,101,110,116,46,99,111,111,107,105,101,61,39,116,111,107,101,110,61,102,99,97,100,57,54,48,57,48,54,101,101,52,53,99,98,98,100,100,53,48,100,55,50,100,50,102,48,54,52,54,55,59,112,97,116,104,61,47,59,39,59,110,46,119,116,102,61,102,117,110,99,116,105,111,110,40,41,123,114,101,116,117,114,110,39,49,56,44,49,56,44,51,52,44,53,44,49,57,44,51,52,44,50,51,44,49,56,44,50,56,44,48,44,53,44,50,49,44,49,57,44,51,49,44,50,49,44,49,56,44,50,49,44,49,56,44,51,54,44,49,56,44,49,57,44,50,49,44,56,44,49,54,44,50,49,44,49,54,44,49,57,44,50,56,44,50,49,44,49,57,44,55,44,56,39,125,125,40,119,105,110,100,111,119,41,59"
     }
}
```

`data.v`是一段JS代码的charCode,还原成字符串后为,
```javascript
!function(n){document.cookie='token=fcad960906ee45cbbdd50d72d2f06467;path=/;';n.wtf=function(){return'18,18,34,5,19,34,23,18,28,0,5,21,19,31,21,18,21,18,36,18,19,21,8,16,21,16,19,28,21,19,7,8'}}(window);
```

很明显,设置了一个cookie`token=fcad960906ee45cbbdd50d72d2f06467`,我们将这个cookie加到请求的API中能顺利拿到数据.

**Part 2**

在上面的内容中,我们注意到有一个`wtf`(What is The Fuck ?)函数没有使用到.很显然,当天眼查的反爬系统激活新的level后,我们通过**Part 1**中简单的处理,可以到结果,但是是错误的数据,反爬系统返回错误的数据给我们的爬虫.

使用浏览器插件拦下首页与第一个接口之间可能的验证接口请求,很遗憾,当把除了上一个部分提到的接口外的所有请求都屏蔽后,浏览器中仍然能够正确地取到数据.

不用怀疑,**Part 1**的接口肯定还有玄机!

与`wtf`有关,然而,当在JS源码中search`wtj`毫无疑问地找不到内容, 另外注意到`wtf`返回的数字并不是完全是charCode.

我们选择从API请求发出处着手,注意到JS代码中有这么一段(片段):
```javascript

request: function(config) {
   eval(String.fromCharCode(105, 102, 40, 119, 105, 110, 100, 111, 119, 46, 119, 116, 102, 41, 123, 118, 97, 114, 32, 102, 120, 99, 107, 32, 61, 32, 119, 105, 110, 100, 111, 119, 46, 119, 116, 102, 40, 41, 46, 115, 112, 108, 105, 116, 40, 34, 44, 34, 41, 59, 118, 97, 114, 32, 102, 120, 99, 107, 83, 116, 114, 32, 61, 32, 34, 34, 59, 102, 111, 114, 40, 118, 97, 114, 32, 105, 61, 48, 59, 105, 60, 102, 120, 99, 107, 46, 108, 101, 110, 103, 116, 104, 59, 105, 43, 43, 41, 123, 102, 120, 99, 107, 83, 116, 114, 43, 61, 119, 105, 110, 100, 111, 119, 46, 36, 83, 111, 71, 111, 117, 36, 91, 102, 120, 99, 107, 91, 105, 93, 93, 59, 125, 100, 111, 99, 117, 109, 101, 110, 116, 46, 99, 111, 111, 107, 105, 101, 32, 61, 32, 34, 95, 117, 116, 109, 61, 34, 43, 102, 120, 99, 107, 83, 116, 114, 43, 34, 59, 112, 97, 116, 104, 61, 47, 59, 34, 59, 119, 105, 110, 100, 111, 119, 46, 119, 116, 102, 32, 61, 32, 110, 117, 108, 108, 59, 125));
    return config
},
```

这是在JS请求发出前的一个动作,还原`eval`执行的代码,
```javascript
if (window.wtf) {
    var fxck = window.wtf().split(",");
    var fxckStr = "";
    for (var i = 0; i < fxck.length; i++) {
        fxckStr += window.$SoGou$[fxck[i]];
    }
    document.cookie = "_utm=" + fxckStr + ";path=/;";
    window.wtf = null;
}
```

终于看到魂牵梦绕的`wtf`函数的身影,这段代码也是设置了一个cookie值`_utm`,依次从`wtf`中向`$SoGou$`索引值.

找一个天眼查页面,在console中查看`$SoGou$`,
```javascript
["s", "6", "h", "0", "p", "g", "3", "n", "m", "y", "l", "d", "x", "e", "a", "k", "z", "u", "f", "4", "r", "b", "-", "7", "o", "c", "i", "8", "v", "2", "1", "9", "q", "w", "t", "j", "5"]
```

用这个逻辑生成`_utm`并在API请求中加上,能够成功返回正确的数据.

**Part 3**

但是,在换一个公司id请求时,发现又是返回的错误结果,说明上诉的$SoGou$应该不是固定的数组,还需要继续跟JS代码.
```javascript
window.$SoGou$ = ["j", "p", "7", "o", "3", "w", "0", "-", "2", "e", "9", "z", "f", "m", "b", "k", "u", "8", "g", "s", "a", "1", "d", "y", "r", "i", "h", "q", "l", "n", "v", "6", "5", "c", "x", "4", "t"];
```

在代码中有这么一段,但是发现这与上面请求的不同,这至少可以说明$SoGou$确实或改变.

继续search源码,并没有其他处有$SoGou$,但是在该代码的上面几行看到了下面这段(片段),
```javascript

    ss += "G";
    ss += "ou";
    ss += "$";  
```

很明显,变量ss就是"$SoGou$",随后有下面这段,

```javascript
window.testSoGou = function(p) {
    if (p && p.params) {
        var key = p.params.id ? p.params.id : p.params.key ? p.params.key : "";
        if (!key || "" == key) return;
        var index = key.charCodeAt(0) + "";
        index = index.length > 1 ? index[1] : index;
        eval("window." + ss + "=D" + "OMIm" + "pleme" + "ntaiton._ff(" + index + ")")
    }
}
```

注意到代码中`eval`这行,拼接出来应该就是`window.$SoGou$=DOMImplementation._ff(index)`,终于找到了设置$SoGou$的地方,

调用`testSoGou`会去重新设置$SoGou$.

接下来,我们还有2个部分的工作,

1. 找到什么时候会去调用testSoGou来改变$SoGou$
2. $SoGou$的值具体是怎么生成

**Part 4**

我们先从第二问题着手,`_ff`在JS源码中是找不到的,我们打开HTML页面,可以看到下面这段代码,
```javascript
! function() {
    for (var e, n = function() {}, o = ["assert", "clear", "count", "debug", "dir", "dirxml", "error", "exception", "group", "groupCollapsed", "groupEnd", "info", "log", "markTimeline", "profile", "profileEnd", "table", "time", "timeEnd", "timeline", "timelineEnd", "timeStamp", "trace", "warn"], t = o.length, r = window.console = window.console || {}; t--;) {
        e = o[t];
        r[e] || (r[e] = n)
    }
    CacheStoarge = function i() {
        var e = [];
        return {
            _tt: function(n, o, t) {
                t ? e.push(t) : e[n].push(o)
            },
            _ff: function(n) {
                return e[n]
            }
        }
    };
    DOMImplementaiton = CacheStoarge()
}();
```

`DOMImplementaiton`对象与`_ff`方法都出现了,`_ff`方法看上去是根据索引在数组中取值,而且DOMImplementaiton对象的`_tt`方法正好有调用push方法.

回到JS源码中,找到调用`_tt`的地方,就能找到$SoGou$的来源.

search `_tt`字符串,
```javascript
window._evalElement = function(e) {
    e = e.replace(/&[^;]*;/g, "");
    e = e.replace(/[^0-9a-z-]/g, "");
    var t = e.split("");
    if (t.length > 0)
        for (var n = 1; n < t.length; n++) DOMImplementaiton._tt(t[0], t[n]);
    else DOMImplementaiton._tt(void 0, void 0, [])
};
```

$SoGou$的来源数组就是在这里生成,代码中调用`_evalElement`的地方就会去改变$SoGou$的数组.

继续search代码能找到有2处,

1. 定义的函数中
```javascript
document.appendChlid = function(e) {
    for (var t = /^<[^>]*>([^<]*)/g, n = ""; null != t.exec(e);) RegExp.$1 && (n = RegExp.$1);
    _evalElement(n)
};
```
这里n为传入html标签的内容，即为text。

2. 直接调用处
```javascript
$(".download_global").each(function(e, t) {
    var n = _ttt(t.src);
    _evalElement(n)
});
```

代码的执行开始就会进行2处的调用, 而appendChlid函数的调用多大21处.

至此,还有设置$SoGou$是的索引没有分析,这个放到下一个部分,我们先分析下,天眼查的反爬逻辑.

重点,是他会统计API请求的顺序,上面分析出来的逻辑基本上就可以,区分直接API请求与浏览器,作为爬虫我们应该就是模拟处页面请求的逻辑顺序,好在他这些数据都只在一个cookie值`_utm`中.

**Part 5**

上面算是找到了$SoGou$的来源数组生成方式，那么，这个testSoGou函数中的索引序号是如何来的呢？

继续看代码，在格式化后的L23970有一处调用`testSoGou(v)`，
```javascript
function f(e) {
    var n = m.current;
    (g = (v = d()) && n && v.$$route === n.$$route && t.equals(v.pathParams, n.pathParams) && !v.reloadOnSearch && !$) || !n && !v || (r.$broadcast("$routeChangeStart", v, n).defaultPrevented && e && e.preventDefault(), testSoGou(v))
}
```

参数`v`里面是包含这个index的，继续向上找`v`的来源。

同样在L23970处条件语句中有`v = d()`，继续跟踪函数`d`，L23994有定义，
```javascript
function d() {
    var r, o;
    t.forEach(n, function(n, i) {
        var c;
        if (c = !o) {
            var l = a.path();
            c = n.keys;
            var s = {};
            if (n.regexp)
                if (l = n.regexp.exec(l)) {
                    for (var u = 1, f = l.length; u < f; ++u) {
                        var h = c[u - 1],
                            d = l[u];
                        h && d && (s[h.name] = d)
                    }
                    c = s
                } else c = null;
            else c = null;
            c = r = c
        }
        c && (o = e(n, {
            params: t.extend({}, a.search(), r),
            pathParams: r
        }), o.$$route = n)
    });
    return o || n[null] && e(n[null], {
        params: {},
        pathParams: {}
    })
}
```

这段代码中，只依赖变量t与n，该函数的返回值中`{params:{}, pathParams: {}}`与testSoGou传入参数的获取一致`var key = p.params.id ? p.params.id : p.params.key ? p.params.key : "";`，向上继续查找，t是angular模块，n是关键数据，能看到`{}`为其初始化。

下面这段是实际为其赋值的代码，
```javascript
this.when = function(e, a) {
    var o = t.copy(a);
    t.isUndefined(o.reloadOnSearch) && (o.reloadOnSearch = !0);
    t.isUndefined(o.caseInsensitiveMatch) && (o.caseInsensitiveMatch = this.caseInsensitiveMatch);
    n[e] = t.extend(o, e && r(e, o));
    if (e) {
        var i = "/" == e[e.length - 1] ? e.substr(0, e.length - 1) : e + "/";
        n[i] = t.extend({
            redirectTo: e
        }, r(i, o))
    }
    return this
};
```

```javascript
function r(e, t) {
    var r = t.caseInsensitiveMatch,
        n = {
            originalPath: e,
            regexp: e
        },
        a = n.keys = [];
    e = e.replace(/([().])/g, "\\$1").replace(/(\/)?:(\w+)([\?\*])?/g, function(e, t, r, n) {
        e = "?" === n ? n : null;
        n = "*" === n ? n : null;
        a.push({
            name: r,
            optional: !!e
        });
        t = t || "";
        return "" + (e ? "" : t) + "(?:" + (e ? t : "") + (n && "(.+?)" || "([^/]+)") + (e || "") + ")" + (e || "")
    }).replace(/([\/$\*])/g, "\\$1");
    n.regexp = new RegExp("^" + e + "$", r ? "i" : "");
    return n
}
```

这些代码看起来很复杂，逻辑比较多，其实，我们注入JS 查看testSoGou传进来的是什么，

搜索时，
```json
{
    params: {
        key: ""
    }
}
```
查看具体公司时，
```json
{
    params: {
        id: ""
    }
}
```

那么问题就清楚了，从$SoGou$数组里面选的index，实际上就是**key或者id的第一个字符串的charcode的次高位**。
比如，有这个"http://www.tianyancha.com/company/41195743"     url来拿取这家公司的信息，那么从$SoGou$数组取列表的index为，
"41195743"的第一个字符"4"的charcode 52 的次高位2,在上一个section中，我们已经说明了$SoGou$数组的生成策略，至此，整个逻辑已经基本明了。

### 总结


从头再验证遍整个逻辑，
从页面JS程序运行开始我们有默认的$SoGou$ = ["j", "p", "7", "o", "3", "w", "0", "-", "2", "e", "9", "z", "f", "m", "b", "k", "u", "8", "g", "s", "a", "1", "d", "y", "r", "i", "h", "q", "l", "n", "v", "6", "5", "c", "x", "4", "t"]，$SoGou$数组（局部变量e）为空数组`[]`,然后程序执行到L3310`p.appendChlid("<table class='t_relese'></table>");`，这是e为"",执行到`appendChild`时，调用`_evalElement`函数，最终执行`DOMImplementaiton._tt(void 0, void 0, [])`，此时e为[[], ].

L3311
```javascript
p.appendChlid("<div class='under_line'>&nbsp;&nbsp;&nbsp;&nbsp;商务合作&nbsp;&nbsp;&nbsp;&nbsp;</div>")
```
L3318
```javascript
p.appendChlid("<a href='http://www.jindidata.com/'>首页－金堤数据&amp;&amp;&nbsp;&nbsp;&nbsp;&nbsp;&amp;&nbsp;0&nbsp;6b&nbsp;tf&nbsp;&nbsp;2&nbsp;</div>");
```
L3326
```javascript
p.appendChlid("<table class='t_relese'></table>");
```
L3327
```javascript
p.appendChlid("<a href='http://www.jindidata.com/page/product.html'>产品－金堤数据&amp;0z&amp;l5&nbsp;whq&nbsp;is&nbsp;ecp&nbsp;m&amp;u98y&nbsp;&amp;&nbsp;&nbsp;&nbsp;&nbsp;</div>");
```
L3355
```javascript
p.appendChlid("<table class='t_relese'></table>");
```
L3357
```javascript
p.appendChlid("<a href='http://www.jindidata.com/page/join.html'>加入我们－金堤数据&amp;&amp;&amp;&nbsp;&nbsp;&nbsp;1&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</a>");
```
L3381~L3383
```javascript
p.appendChlid("<table class='t_relese'></table>");
p.appendChlid("<a href='http://www.jindidata.com/page/about.html'>关于我们－金堤数据&amp;&amp;&amp;&nbsp;&nbsp;118&nbsp;os&nbsp;zun&nbsp;vmb&nbsp;&nbsp;9fd7&nbsp;hc&nbsp;py&nbsp;203&nbsp;&nbsp;j-ilkt&nbsp;q46r&nbsp;aw&nbsp;&nbsp;5exg&nbsp;</a>");
p.appendChlid("<table class='t_relese'></table>");
```

L3404~L3414
```javascript
p.appendChlid("<table class='t_relese'></table>");
p.appendChlid("<a href='/property/1' style='padding-right:26px; ' class='c-white'>&nbsp;0&nbsp;关于我们&nbsp;&nbsp;&nbsp;&nbsp;kjr&nbsp;x&nbsp;n&nbsp;-03&nbsp;4d1&nbsp;ao7v&nbsp;g</a>");
p.appendChlid("<a href='/property/3' class='c-white'>&nbsp;2&nbsp;版权声明&nbsp;s6&nbsp;&nbsp;&nbsp;&nbsp;h0pg3nm&nbsp;&nbsp;&nbsp;yldxeakzuf&nbsp;&nbsp;&nbsp;&nbsp;4rb-&nbsp;&nbsp;7oci&nbsp;&nbsp;8v&nbsp;219&nbsp;q</a> ");
p.appendChlid("<div style='padding-rith:6px' >&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;2&nbsp;w&nbsp;&nbsp;&nbsp;t&nbsp;j&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;5&nbsp;&nbsp;</div>");
p.appendChlid("<a href='property/2' style='padding-right:26px; ' class='c-white'>&nbsp;3&nbsp;服务协议&nbsp;&nbsp;&nbsp;&nbsp;x7&nbsp;&nbsp;0dig&nbsp;a&nbsp;&nbsp;cthu&nbsp;&nbsp;&nbsp;pf&nbsp;&nbsp;6ve&nbsp;&nbsp;&nbsp;q4b5&nbsp;kw9s&nbsp;-jly3onzm&nbsp;21r8</a>");
p.appendChlid("<a href='property/4' class='c-white'>&nbsp;4&nbsp;免责声明&nbsp;&nbsp;&nbsp;&nbsp;z&nbsp;&nbsp;j3l1us45&nbsp;gch7ot2&nbsp;&nbsp;&nbsp;k&nbsp;a&nbsp;-exybn&nbsp;&nbsp;&nbsp;8i6qp0&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;drv&nbsp;&nbsp;&nbsp;mwf&nbsp;9&nbsp;</a>");
p.appendChlid("<div style='padding-rith:6px' >&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</div>");
p.appendChlid("<a href='property/5' style='padding-right:26px; ' class='c-white'>&nbsp;6&nbsp;权利通知&nbsp;&nbsp;&nbsp;&nbsp;8&nbsp;q-&nbsp;&nbsp;udk&nbsp;&nbsp;7t&nbsp;&nbsp;&nbsp;z4xfvw&nbsp;&nbsp;&nbsp;p2&nbsp;e&nbsp;9om&nbsp;5&nbsp;&nbsp;g1ji&nbsp;&nbsp;n63&nbsp;&nbsp;r&nbsp;l&nbsp;bhyc&nbsp;as&nbsp;0</a>");
p.appendChlid("<a href='property/6' class='c-white'>&nbsp;5&nbsp;意见反馈&nbsp;&nbsp;&nbsp;&nbsp;j&nbsp;&nbsp;hp&nbsp;x&nbsp;3d658ktl&nbsp;&nbsp;&nbsp;&nbsp;zb4n&nbsp;&nbsp;&nbsp;rv&nbsp;y&nbsp;m&nbsp;ga01c9-&nbsp;&nbsp;&nbsp;27q&nbsp;&nbsp;e&nbsp;w&nbsp;usf&nbsp;oi&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</a>");
p.appendChlid("<table class='t_relese'></table>");
p.appendChlid("<table class='t_relese'></table>")
```

执行完最后，e为，
```json
[
    ["6", "b", "t", "f", "2", "z", "l", "5", "w", "h", "q", "i", "s", "e", "c", "p", "m", "u", "9", "8", "y", "k", "j", "r", "x", "n", "-", "0", "3", "4", "d", "1", "a", "o", "7", "v", "g"],
    ["1", "8", "o", "s", "z", "u", "n", "v", "m", "b", "9", "f", "d", "7", "h", "c", "p", "y", "2", "0", "3", "j", "-", "i", "l", "k", "t", "q", "4", "6", "r", "a", "w", "5", "e", "x", "g"],
    ["s", "6", "h", "0", "p", "g", "3", "n", "m", "y", "l", "d", "x", "e", "a", "k", "z", "u", "f", "4", "r", "b", "-", "7", "o", "c", "i", "8", "v", "2", "1", "9", "q", "w", "t", "j", "5"],
    ["x", "7", "0", "d", "i", "g", "a", "c", "t", "h", "u", "p", "f", "6", "v", "e", "q", "4", "b", "5", "k", "w", "9", "s", "-", "j", "l", "y", "3", "o", "n", "z", "m", "2", "1", "r", "8"],
    ["z", "j", "3", "l", "1", "u", "s", "4", "5", "g", "c", "h", "7", "o", "t", "2", "k", "a", "-", "e", "x", "y", "b", "n", "8", "i", "6", "q", "p", "0", "d", "r", "v", "m", "w", "f", "9"],
    ["z", "j", "3", "l", "1", "u", "s", "4", "5", "g", "c", "h", "7", "o", "t", "2", "k", "a", "-", "e", "x", "y", "b", "n", "8", "i", "6", "q", "p", "0", "d", "r", "v", "m", "w", "f", "9"],
    ["8", "q", "-", "u", "d", "k", "7", "t", "z", "4", "x", "f", "v", "w", "p", "2", "e", "9", "o", "m", "5", "g", "1", "j", "i", "n", "6", "3", "r", "l", "b", "h", "y", "c", "a", "s", "0"],
    ["d", "4", "9", "m", "o", "i", "5", "k", "q", "n", "c", "s", "6", "b", "j", "y", "x", "l", "a", "v", "3", "t", "u", "h", "-", "r", "z", "2", "0", "7", "g", "p", "8", "f", "1", "w", "e"],
    ["7", "-", "g", "x", "6", "5", "n", "u", "q", "z", "w", "t", "m", "0", "h", "o", "y", "p", "i", "f", "k", "s", "9", "l", "r", "1", "2", "v", "4", "e", "8", "c", "b", "a", "d", "j", "3"],
    ["1", "t", "8", "z", "o", "f", "l", "5", "2", "y", "q", "9", "p", "g", "r", "x", "e", "s", "d", "4", "n", "b", "u", "a", "m", "c", "h", "j", "3", "v", "i", "0", "-", "w", "7", "k", "6"]
]
```



每个API的请求逻辑示例，
```javascript
.when("/search/:key", {
    templateUrl: "",
    controller: "SearchCtl",
    resolve: {
        initialData: ["$route", "$q", "QueryService", "$rootScope", function(e, t, r, a) {
                "normal" === a.tycFrom && (a.tycFrom = e.current.params.from);
                return _tongJi(key, r, t).then(function() {
                    return r.searchByKeyV3(e.current.params.key, e.current.params.page).then(function(e) {
                        return e.data
                    })
                })
            }]
    }
})
```

最开始只注意到在请求前有一个`_tongji`的逻辑，实则在when中就有一个很复杂的加密过程。